<a href="https://colab.research.google.com/github/srikanthgr/pyspark-python/blob/main/pyspark_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.6.tgz
!tar -xvf spark-2.4.3-bin-hadoop2.6.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.6"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
df = spark.createDataFrame([{"Google": "Colab","Spark": "Scala"} ,{"Google": "Dataproc","Spark":"Python"}])
df.show()


/content/spark-2.4.3-bin-hadoop2.6/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------+------+
|  Google| Spark|
+--------+------+
|   Colab| Scala|
|Dataproc|Python|
+--------+------+



In [3]:
!ls

sample_data  spark-2.4.3-bin-hadoop2.6	spark-2.4.3-bin-hadoop2.6.tgz


In [4]:
!wget https://raw.githubusercontent.com/futurexskill/bidata/master/retailstore.csv

--2021-03-04 10:23:01--  https://raw.githubusercontent.com/futurexskill/bidata/master/retailstore.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279 [text/plain]
Saving to: ‘retailstore.csv’

retailstore.csv     100%[===================>]     279  --.-KB/s    in 0s      

2021-03-04 10:23:01 (19.7 MB/s) - ‘retailstore.csv’ saved [279/279]



In [5]:
!ls

retailstore.csv  spark-2.4.3-bin-hadoop2.6
sample_data	 spark-2.4.3-bin-hadoop2.6.tgz


In [6]:
customerDF = spark.read.csv("retailstore.csv", header=True)
type(customerDF)

pyspark.sql.dataframe.DataFrame

In [7]:
customerDF.show(4)

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 18| 20000|  Male|Germany|        N|
| 19| 22000|Female| France|        N|
| 20| 24000|Female|England|        N|
| 21|  null|  Male|England|        N|
+---+------+------+-------+---------+
only showing top 4 rows



In [9]:
customerDF.describe().show()

+-------+-----------------+-----------------+------+-------+---------+
|summary|              Age|           Salary|Gender|Country|Purchased|
+-------+-----------------+-----------------+------+-------+---------+
|  count|                9|                8|    10|     10|       10|
|   mean|22.11111111111111|          31875.0|  null|   null|     null|
| stddev|2.934469476943168|9818.895777311942|  null|   null|     null|
|    min|               18|            20000|Female|England|        N|
|    max|               27|            50000|  Male|Germany|        Y|
+-------+-----------------+-----------------+------+-------+---------+



In [10]:
customerDF.select("country").show()

+-------+
|country|
+-------+
|Germany|
| France|
|England|
|England|
| France|
|England|
|Germany|
| France|
|Germany|
| France|
+-------+



In [11]:
customerDF.groupby("country").count().show()

+-------+-----+
|country|count|
+-------+-----+
|Germany|    3|
| France|    4|
|England|    3|
+-------+-----+



In [12]:
customerDF.groupBy("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|    5|
|  Male|    5|
+------+-----+



In [16]:
customerDF.createOrReplaceTempView("customer")
results = spark.sql("SELECT * FROM customer")
type(results)
results.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [17]:
new_results = spark.sql("SELECT * FROM customer where age > 22").show()

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 23| 35000|Female|England|        N|
| 24|  null|  Male|Germany|        N|
| 25| 32000|Female| France|        Y|
| 27| 37000|Female| France|        N|
+---+------+------+-------+---------+



In [18]:
new_results = spark.sql("SELECT * FROM customer where age > 22").select("salary").show()

+------+
|salary|
+------+
| 35000|
|  null|
| 32000|
| 37000|
+------+



In [19]:
customerDF.filter("salary > 30000").show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [21]:
customerDF.groupby("gender").agg({"salary":"avg", "age":"max"}).show()

+------+-----------+--------+
|gender|avg(salary)|max(age)|
+------+-----------+--------+
|Female|    30000.0|      27|
|  Male|    35000.0|      24|
+------+-----------+--------+



In [22]:
customerDF.select(["age", "salary"]).show()

+----+------+
| age|salary|
+----+------+
|  18| 20000|
|  19| 22000|
|  20| 24000|
|  21|  null|
|  22| 50000|
|  23| 35000|
|  24|  null|
|  25| 32000|
|null| 35000|
|  27| 37000|
+----+------+



In [23]:
customerDF.withColumn('doublesalary', customerDF['salary']*2).show()

+----+------+------+-------+---------+------------+
| Age|Salary|Gender|Country|Purchased|doublesalary|
+----+------+------+-------+---------+------------+
|  18| 20000|  Male|Germany|        N|     40000.0|
|  19| 22000|Female| France|        N|     44000.0|
|  20| 24000|Female|England|        N|     48000.0|
|  21|  null|  Male|England|        N|        null|
|  22| 50000|  Male| France|        Y|    100000.0|
|  23| 35000|Female|England|        N|     70000.0|
|  24|  null|  Male|Germany|        N|        null|
|  25| 32000|Female| France|        Y|     64000.0|
|null| 35000|  Male|Germany|        N|     70000.0|
|  27| 37000|Female| France|        N|     74000.0|
+----+------+------+-------+---------+------------+



In [24]:
customerDF.withColumnRenamed('salary','income').show()

+----+------+------+-------+---------+
| Age|income|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [25]:
customerDF.filter("salary > 30000").select('age').show()

+----+
| age|
+----+
|  22|
|  23|
|  25|
|null|
|  27|
+----+



In [26]:
customerDF.filter("salary > 30000").select('age', 'country').show()

+----+-------+
| age|country|
+----+-------+
|  22| France|
|  23|England|
|  25| France|
|null|Germany|
|  27| France|
+----+-------+



In [27]:
customerDF.filter(customerDF['salary']> 30000).show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [31]:
customerDF.filter((customerDF['salary'] > 30000) & ~(customerDF['age'] < 22)).show()

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 22| 50000|  Male| France|        Y|
| 23| 35000|Female|England|        N|
| 25| 32000|Female| France|        Y|
| 27| 37000|Female| France|        N|
+---+------+------+-------+---------+



In [34]:
from pyspark.sql.functions import countDistinct, avg, stddev
customerDF.select(countDistinct("country")).show()

+-----------------------+
|count(DISTINCT country)|
+-----------------------+
|                      3|
+-----------------------+



In [35]:
customerDF.select(avg('salary')).show()

+-----------+
|avg(salary)|
+-----------+
|    31875.0|
+-----------+



In [36]:
customerDF.select(countDistinct("country").alias("Distinct Countries")).show()

+------------------+
|Distinct Countries|
+------------------+
|                 3|
+------------------+



In [37]:
customerDF.select(avg('age')).show()

+-----------------+
|         avg(age)|
+-----------------+
|22.11111111111111|
+-----------------+



In [38]:
customerDF.select(stddev("salary")).show()

+-------------------+
|stddev_samp(salary)|
+-------------------+
|  9818.895777311942|
+-------------------+



In [39]:
customerDF.select(stddev("age")).show()

+-----------------+
| stddev_samp(age)|
+-----------------+
|2.934469476943168|
+-----------------+



In [40]:
from pyspark.sql.functions import format_number

salary_std = customerDF.select(stddev("salary").alias('std'))

salary_std.show()


+-----------------+
|              std|
+-----------------+
|9818.895777311942|
+-----------------+



In [41]:
salary_std.select(format_number('std',2)).show()

+---------------------+
|format_number(std, 2)|
+---------------------+
|             9,818.90|
+---------------------+



In [42]:
customerDF.orderBy(customerDF["salary"].desc()).show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  22| 50000|  Male| France|        Y|
|  27| 37000|Female| France|        N|
|  23| 35000|Female|England|        N|
|null| 35000|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|  20| 24000|Female|England|        N|
|  19| 22000|Female| France|        N|
|  18| 20000|  Male|Germany|        N|
|  21|  null|  Male|England|        N|
|  24|  null|  Male|Germany|        N|
+----+------+------+-------+---------+



In [43]:
customerDF.orderBy("salary").show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  21|  null|  Male|England|        N|
|  24|  null|  Male|Germany|        N|
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  23| 35000|Female|England|        N|
|  27| 37000|Female| France|        N|
|  22| 50000|  Male| France|        Y|
+----+------+------+-------+---------+



In [45]:
customerDF.na.drop().show()

+---+------+------+-------+---------+
|Age|Salary|Gender|Country|Purchased|
+---+------+------+-------+---------+
| 18| 20000|  Male|Germany|        N|
| 19| 22000|Female| France|        N|
| 20| 24000|Female|England|        N|
| 22| 50000|  Male| France|        Y|
| 23| 35000|Female|England|        N|
| 25| 32000|Female| France|        Y|
| 27| 37000|Female| France|        N|
+---+------+------+-------+---------+



In [46]:
customerDF.na.fill('NEW VALUE').show()

+---------+---------+------+-------+---------+
|      Age|   Salary|Gender|Country|Purchased|
+---------+---------+------+-------+---------+
|       18|    20000|  Male|Germany|        N|
|       19|    22000|Female| France|        N|
|       20|    24000|Female|England|        N|
|       21|NEW VALUE|  Male|England|        N|
|       22|    50000|  Male| France|        Y|
|       23|    35000|Female|England|        N|
|       24|NEW VALUE|  Male|Germany|        N|
|       25|    32000|Female| France|        Y|
|NEW VALUE|    35000|  Male|Germany|        N|
|       27|    37000|Female| France|        N|
+---------+---------+------+-------+---------+

